In [ ]:
def setupSpark():
  # Spark needs to run with Java 8 ... 
  !pip install -q findspark
  !apt-get install openjdk-8-jdk-headless > /dev/null
  !echo 2 | update-alternatives --config java > /dev/null
  !java -version
  import os, findspark
  os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
  # !echo JAVA_HOME=$JAVA_HOME
  !pip install -q pyspark
  findspark.init(spark_home='/usr/local/lib/python3.7/dist-packages/pyspark')
  !pyspark --version


setupSpark()

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 212.4 MB 54 kB/s 
     |████████████████████████████████| 198 kB 54.0 MB/s 
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [ ]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
conf=SparkConf()
SpContext=SparkContext(conf=conf)
SpSession=SparkSession(SpContext)

## Load Data

In [ ]:

#Load the CSV file into a RDD
autoData = SpContext.textFile('/content/auto-mpg.csv')
autoData.cache()
autoData.take(5)
#Remove the first line (contains headers)
dataLines = autoData.filter(lambda x: "CYLINDERS" not in x)
dataLines.count()

398

In [ ]:
autoData.collect()

['MPG,CYLINDERS,DISPLACEMENT,HORSEPOWER,WEIGHT,ACCELERATION,MODEL YEAR,ORIGIN,CAR NAME',
 '18,8,307,130,3504,12,70,1,chevrolet chevelle malibu',
 '15,8,350,165,3693,11.5,70,1,buick skylark 320',
 '18,8,318,150,3436,11,70,1,plymouth satellite',
 '16,8,304,150,3433,12,70,1,amc rebel sst',
 '17,8,302,140,3449,10.5,70,1,ford torino',
 '15,8,429,198,4341,10,70,1,ford galaxie 500',
 '14,8,454,220,4354,9,70,1,chevrolet impala',
 '14,8,440,215,4312,8.5,70,1,plymouth fury iii',
 '14,8,455,225,4425,10,70,1,pontiac catalina',
 '15,8,390,190,3850,8.5,70,1,amc ambassador dpl',
 '15,8,383,170,3563,10,70,1,dodge challenger se',
 "14,8,340,160,3609,8,70,1,plymouth 'cuda 340",
 '15,8,400,150,3761,9.5,70,1,chevrolet monte carlo',
 '14,8,455,225,3086,10,70,1,buick estate wagon (sw)',
 '24,4,113,95,2372,15,70,3,toyota corona mark ii',
 '22,6,198,95,2833,15.5,70,1,plymouth duster',
 '18,6,199,97,2774,15.5,70,1,amc hornet',
 '21,6,200,85,2587,16,70,1,ford maverick',
 '27,4,97,88,2130,14.5,70,3,datsun pl510'

## Cleanup Data

In [ ]:
from pyspark.sql import Row

#Use default for average HP
avgHP =SpContext.broadcast(80.0)

#Function to cleanup Data
def CleanupData( inputStr) :
    global avgHP
    attList=inputStr.split(",")
    
    #Replace ? values with a normal value
    hpValue = attList[3]
    if hpValue == "?":
        hpValue=avgHP.value
       
    #Create a row with cleaned up and converted data
    values= Row(     MPG=float(attList[0]),\
                     CYLINDERS=float(attList[1]), \
                     DISPLACEMENT=float(attList[2]), 
                     HORSEPOWER=float(hpValue),\
                     WEIGHT=float(attList[4]), \
                     ACCELERATION=float(attList[5]), \
                     MODELYEAR=float(attList[6]),\
                     NAME=attList[7]  ) 
    return values

#Run map for cleanup
autoMap = dataLines.map(CleanupData)
autoMap.cache()
autoMap.take(5)
#Create a Data Frame with the data. 
autoDf = SpSession.createDataFrame(autoMap)
autoDf.show(5)

+----+---------+------------+----------+------+------------+---------+----+
| MPG|CYLINDERS|DISPLACEMENT|HORSEPOWER|WEIGHT|ACCELERATION|MODELYEAR|NAME|
+----+---------+------------+----------+------+------------+---------+----+
|18.0|      8.0|       307.0|     130.0|3504.0|        12.0|     70.0|   1|
|15.0|      8.0|       350.0|     165.0|3693.0|        11.5|     70.0|   1|
|18.0|      8.0|       318.0|     150.0|3436.0|        11.0|     70.0|   1|
|16.0|      8.0|       304.0|     150.0|3433.0|        12.0|     70.0|   1|
|17.0|      8.0|       302.0|     140.0|3449.0|        10.5|     70.0|   1|
+----+---------+------------+----------+------+------------+---------+----+
only showing top 5 rows



In [ ]:
autoDf.collect()

[Row(MPG=18.0, CYLINDERS=8.0, DISPLACEMENT=307.0, HORSEPOWER=130.0, WEIGHT=3504.0, ACCELERATION=12.0, MODELYEAR=70.0, NAME='1'),
 Row(MPG=15.0, CYLINDERS=8.0, DISPLACEMENT=350.0, HORSEPOWER=165.0, WEIGHT=3693.0, ACCELERATION=11.5, MODELYEAR=70.0, NAME='1'),
 Row(MPG=18.0, CYLINDERS=8.0, DISPLACEMENT=318.0, HORSEPOWER=150.0, WEIGHT=3436.0, ACCELERATION=11.0, MODELYEAR=70.0, NAME='1'),
 Row(MPG=16.0, CYLINDERS=8.0, DISPLACEMENT=304.0, HORSEPOWER=150.0, WEIGHT=3433.0, ACCELERATION=12.0, MODELYEAR=70.0, NAME='1'),
 Row(MPG=17.0, CYLINDERS=8.0, DISPLACEMENT=302.0, HORSEPOWER=140.0, WEIGHT=3449.0, ACCELERATION=10.5, MODELYEAR=70.0, NAME='1'),
 Row(MPG=15.0, CYLINDERS=8.0, DISPLACEMENT=429.0, HORSEPOWER=198.0, WEIGHT=4341.0, ACCELERATION=10.0, MODELYEAR=70.0, NAME='1'),
 Row(MPG=14.0, CYLINDERS=8.0, DISPLACEMENT=454.0, HORSEPOWER=220.0, WEIGHT=4354.0, ACCELERATION=9.0, MODELYEAR=70.0, NAME='1'),
 Row(MPG=14.0, CYLINDERS=8.0, DISPLACEMENT=440.0, HORSEPOWER=215.0, WEIGHT=4312.0, ACCELERATION=8.

## Perform Data Analytics

In [ ]:
#See descriptive analytics.
autoDf.select("MPG","CYLINDERS").describe().show()


#Find correlation between predictors and target
for i in autoDf.columns:
    if not( isinstance(autoDf.select(i).take(1)[0][0], str)) :
        print( "Correlation to MPG for ", i, autoDf.stat.corr('MPG',i))

+-------+-----------------+------------------+
|summary|              MPG|         CYLINDERS|
+-------+-----------------+------------------+
|  count|              398|               398|
|   mean|23.51457286432161| 5.454773869346734|
| stddev|7.815984312565783|1.7010042445332125|
|    min|              9.0|               3.0|
|    max|             46.6|               8.0|
+-------+-----------------+------------------+

Correlation to MPG for  MPG 1.0
Correlation to MPG for  CYLINDERS -0.7753962854205547
Correlation to MPG for  DISPLACEMENT -0.804202824805898
Correlation to MPG for  HORSEPOWER -0.7746308409203809
Correlation to MPG for  WEIGHT -0.8317409332443345
Correlation to MPG for  ACCELERATION 0.42028891210165126
Correlation to MPG for  MODELYEAR 0.57926713308331


## Prepare data for ML

In [ ]:
#Transform to a Data Frame for input to Machine Learing
#Drop columns that are not required (low correlation)

from pyspark.ml.linalg import Vectors
def transformToLabeledPoint(row) :
    lp = ( row["MPG"], Vectors.dense([row["ACCELERATION"],\
                        row["DISPLACEMENT"], \
                        row["WEIGHT"]]))
    return lp
    
autoLp = autoMap.map(transformToLabeledPoint)
autoDF = SpSession.createDataFrame(autoLp,["label", "features"])
autoDF.select("label","features").show(10)

+-----+-------------------+
|label|           features|
+-----+-------------------+
| 18.0|[12.0,307.0,3504.0]|
| 15.0|[11.5,350.0,3693.0]|
| 18.0|[11.0,318.0,3436.0]|
| 16.0|[12.0,304.0,3433.0]|
| 17.0|[10.5,302.0,3449.0]|
| 15.0|[10.0,429.0,4341.0]|
| 14.0| [9.0,454.0,4354.0]|
| 14.0| [8.5,440.0,4312.0]|
| 14.0|[10.0,455.0,4425.0]|
| 15.0| [8.5,390.0,3850.0]|
+-----+-------------------+
only showing top 10 rows



## Perform Machine Learning

In [ ]:
#Split into training and testing data
(trainingData, testData) = autoDF.randomSplit([0.9, 0.1])
trainingData.count()
testData.count()

#Build the model on training data
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=10)
lrModel = lr.fit(trainingData)

#Print the metrics
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

#Predict on the test data
predictions = lrModel.transform(testData)
predictions.select("prediction","label","features").show()

#Find R2 for Linear Regression
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
evaluator.evaluate(predictions)

Coefficients: [0.1999633852236895,-0.009029512534992417,-0.00651172044023792]
Intercept: 41.56449337514447
+------------------+-----+-------------------+
|        prediction|label|           features|
+------------------+-----+-------------------+
| 9.721628935471024| 11.0|[11.0,429.0,4633.0]|
| 7.416134509565566| 12.0|[11.0,455.0,4951.0]|
|15.632182297858694| 13.0|[11.0,360.0,3821.0]|
|14.305634190829618| 13.0|[13.0,350.0,4100.0]|
|10.607845941587886| 13.0|[13.0,360.0,4654.0]|
|16.593367126217913| 14.0| [8.0,340.0,3609.0]|
|13.395893277619994| 14.0|[12.0,350.0,4209.0]|
|14.620625473710327| 14.0|[13.0,318.0,4096.0]|
|14.107764785527724| 14.0|[13.0,351.0,4129.0]|
|11.995200477434736| 16.0| [9.5,400.0,4278.0]|
| 17.63026331242745| 16.0|[18.5,250.0,3897.0]|
|18.518434193819974| 18.0|[11.0,318.0,3436.0]|
| 28.51942481201457| 20.0|[19.5,140.0,2408.0]|
| 26.86743339708416| 21.0|[14.0,155.0,2472.0]|
|26.112184252829522| 21.0|[16.0,200.0,2587.0]|
|24.194374747653214| 22.0|[14.5,121.0,2945.0]|


0.6296594138333973